In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

 #  **Loading DATABASE (Mapped Positions)**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
data = pd.read_excel('/content/drive/MyDrive/ArticleRecognition/Dataset/ArticleExtraction.xlsx', sheet_name='Mapped Positions')

In [4]:
data.sample(5)

,ProjectLVId,OzNumber,PositionShortText,PositionLongTextTitle,PositionLongText,ProjectArticleId,ProjectStadiumId,ArticleNumber,ArticleSortName,BasePrice,...,SequenceNumber,Comment,RegiePrice,LongText,Preprocessed_PositionShortText,Preprocessed_LongText,MergedPositions,Preprocessed_ArticleSortName,Preprocessed_ArticleDescription,MergedArticles
2573,320032.0,01.12.0030,"HLz 20-2,0, Wohnungstrennwand, d=24",Long Text 1,"<div style=""font-size: 12pt; font-family: Aria...",23101,9093,12-HLZ-0030,HLZ,17568.000,...,2.1,NaN,NaN,Hochlochziegel-Mauerwerk der Innenwand aus Sch...,hlz 20 20 wohnungstrennwand d24,hochlochziegelmauerwerk innenwand schallschutz...,hlz 20 20 wohnungstrennwand d24;hochlochziegel...,hlz,planhlz 12 df 373 / 24 / 249 cm 12 / 20 incl m...,hlz;planhlz 12 df 373 / 24 / 249 cm 12 / 20 in...
1912,319989.0,02.03.0040,"FT-Schacht Falz 1000 h=3,50m NW150 m.Rinne o.A...",Long Text 1,"<div style=""font-size:12pt;font-family:Arial;""...",22934,9092,09-SCH-0200,SCHACHT-FT,1125.000,...,2.5,NaN,NaN,"Schachtbauwerk, rund, lichte Weite DN 1000, au...",ftschacht falz 1000 h350 m nw150 m rinne o abd...,schachtbauwerk rund lichte weite dn 1000 stahl...,ftschacht falz 1000 h350 m nw150 m rinne o abd...,schachtft,schachtring falz 1000 h50 cm steig,schachtft;schachtring falz 1000 h50 cm steig
2111,346789.0,6.003,Betonzweizeiler 16/16/14 auf 10 cm Beton verse...,NaN,NaN,25827,9984,4218003,B 10 KS 0/16 PZ 35 F,1914.134,...,1.2,NaN,NaN,NaN,betonzweizeiler 16 / 16 / 14 10 cm beton verse...,NaN,betonzweizeiler 16 / 16 / 14 10 cm beton verse...,b 10 ks 0 / 16 pz 35 f,beton c 8 / 10 0 / 16,b 10 ks 0 / 16 pz 35 f;beton c 8 / 10 0 / 16
1214,319990.0,02.03.0050,"FT-Schacht Falz 1000 h=4,00m NW150 m.Rinne o.A...",Long Text 1,"<div style=""font-size:12pt;font-family:Arial;""...",22933,9092,09-SCH-0190,SCHACHT-FT,945.000,...,2.4,NaN,NaN,"Schachtbauwerk, rund, lichte Weite DN 1000, au...",ftschacht falz 1000 h400 m nw150 m rinne o abd...,schachtbauwerk rund lichte weite dn 1000 stahl...,ftschacht falz 1000 h400 m nw150 m rinne o abd...,schachtft,schachtkonus falz 1000 / 625 h60 stei,schachtft;schachtkonus falz 1000 / 625 h60 stei
316,344597.0,1.4.4.0160,Stahlbetonrohrleitung DN 400 ohne Erdarbeiten,Long Text 1,"<div style=""font-size:12pt;font-family:Arial;""...",24931,9970,21310400,STB Rohr DN 400,4540.250,...,2.1,NaN,NaN,"Stahlbetonrohrleitung DN 400 liefern, abladen,...",stahlbetonrohrleitung dn 400 erdarbeiten,stahlbetonrohrleitung dn 400 liefern abladen v...,stahlbetonrohrleitung dn 400 erdarbeiten;stahl...,stb rohr dn 400,stb rohr dn 400 kgm,stb rohr dn 400;stb rohr dn 400 kgm


In [5]:
df = data.copy()

In [6]:
df

,ProjectLVId,OzNumber,PositionShortText,PositionLongTextTitle,PositionLongText,ProjectArticleId,ProjectStadiumId,ArticleNumber,ArticleSortName,BasePrice,...,SequenceNumber,Comment,RegiePrice,LongText,Preprocessed_PositionShortText,Preprocessed_LongText,MergedPositions,Preprocessed_ArticleSortName,Preprocessed_ArticleDescription,MergedArticles
0,35340.0,53.38.05.033110,"Zulage Schweißung flache Ecke, PVC-P, SFD 24",Long Text 1,"<div style=""font-size:12pt;font-family:Arial;""...",1742,1103,K141011032000,fugenband,14.3,...,1.1,NaN,NaN,Zulage für die Schweißung einer flachen Ecke e...,zulage schweißung flache ecke pvcp sfd 24,zulage schweißung flachen ecke innenliegenden ...,zulage schweißung flache ecke pvcp sfd 24;zula...,fugenband,schweißung flache ecke pvcp sfd 24,fugenband;schweißung flache ecke pvcp sfd 24
1,35341.0,53.38.05.033120,"Zulage Schweißung senkrechte Ecke, PVC-P, SFD 24",Long Text 1,"<div style=""font-size:12pt;font-family:Arial;""...",1743,1103,K141011032100,fugenband,22.9,...,1.1,NaN,NaN,Zulage für die Schweißung einer senkrechten Ec...,zulage schweißung senkrechte ecke pvcp sfd 24,zulage schweißung senkrechten ecke innenliegen...,zulage schweißung senkrechte ecke pvcp sfd 24;...,fugenband,schweißgsenkrechte ecke pvcp sfd 24,fugenband;schweißgsenkrechte ecke pvcp sfd 24
2,35342.0,53.38.05.033130,"Zulage Schweißung flaches T-Stück, PVC-P, SFD 24",Long Text 1,"<div style=""font-size:12pt;font-family:Arial;""...",1744,1103,K141011032200,fugenband,14.7,...,1.1,NaN,NaN,Zulage für die Schweißung eines flachen T-Stüc...,zulage schweißung flaches tstück pvcp sfd 24,zulage schweißung flachen tstücks innenliegend...,zulage schweißung flaches tstück pvcp sfd 24;z...,fugenband,schweißgflaches tstück pvcp sfd 24,fugenband;schweißgflaches tstück pvcp sfd 24
3,35343.0,53.38.05.033140,"Zulage Schweißung senkrechtes T-Stück, PVC-P, ...",Long Text 1,"<div style=""font-size:12pt;font-family:Arial;""...",1745,1103,K141011032300,fugenband,26.0,...,1.1,NaN,NaN,Zulage für die Schweißung eines senkrechten T-...,zulage schweißung senkrechtes tstück pvcp sfd 24,zulage schweißung senkrechten tstücks innenlie...,zulage schweißung senkrechtes tstück pvcp sfd ...,fugenband,schweißgsenkr tstück pvcp sfd 24,fugenband;schweißgsenkr tstück pvcp sfd 24
4,35344.0,53.38.05.033150,"Zulage Schweißung flache Kreuzung, PVC-P, SFD 24",Long Text 1,"<div style=""font-size:12pt;font-family:Arial;""...",1746,1103,K141011032400,fugenband,27.1,...,1.1,NaN,NaN,Zulage für die Schweißung einer flachen Kreuzu...,zulage schweißung flache kreuzung pvcp sfd 24,zulage schweißung flachen kreuzung innenliegen...,zulage schweißung flache kreuzung pvcp sfd 24;...,fugenband,schweißgflache kreuzung pvcp sfd 24,fugenband;schweißgflache kreuzung pvcp sfd 24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3089,344554.0,1.1.1.0010,Baustelleneinrichtung,Long Text 1,"<div style=""font-size:12pt;font-family:Arial;""...",24972,9970,MK,NaN,100.0,...,4.3,NaN,NaN,"Einrichten, Unterhalten, Abbauen der Baustelle...",baustelleneinrichtung,einrichten unterhalten abbauen baustelleneinri...,baustelleneinrichtung;einrichten unterhalten a...,NaN,materialcontainer,;materialcontainer
3090,344748.0,1.4.3.0010,Verbau (Box/Gleitschienenverbau),Long Text 1,"<div style=""font-size:12pt;font-family:Arial;""...",25047,9971,3303,Verbau Graben bis,1400.0,...,4.1,NaN,NaN,Großflächenverbau (Box/Gleitschienenverbau) li...,verbau box / gleitschienenverbau,großflächenverbau box / gleitschienenverbau li...,verbau box / gleitschienenverbau ;großflächenv...,verbau graben,verbau graben 3 m tiefe,verbau graben ;verbau graben 3 m tiefe
3091,344748.0,1.4.3.0010,Verbau (Box/Gleitschienenverbau),Long Text 1,"<div style=""font-size:12pt;font-family:Arial;""...",25047,9971,3303,Verbau Graben bis,1400.0,...,2.1,NaN,NaN,Großflächenverbau (Box/Gleitschienenverbau) li...,verbau box / gleitschienenverbau,großflächenverbau box / gleitschienenverbau li...,verbau box / gleitschienenver

#**Extracting Useful columns**

In [7]:
df = df[['PositionShortText','LongText','ArticleNumber','Quantity']]

In [8]:
df.head()

,PositionShortText,LongText,ArticleNumber,Quantity
0,"Zulage Schweißung flache Ecke, PVC-P, SFD 24",Zulage für die Schweißung einer flachen Ecke e...,K141011032000,1.0
1,"Zulage Schweißung senkrechte Ecke, PVC-P, SFD 24",Zulage für die Schweißung einer senkrechten Ec...,K141011032100,1.0
2,"Zulage Schweißung flaches T-Stück, PVC-P, SFD 24",Zulage für die Schweißung eines flachen T-Stüc...,K141011032200,1.0
3,"Zulage Schweißung senkrechtes T-Stück, PVC-P, ...",Zulage für die Schweißung eines senkrechten T-...,K141011032300,1.0
4,"Zulage Schweißung flache Kreuzung, PVC-P, SFD 24",Zulage für die Schweißung einer flachen Kreuzu...,K141011032400,1.0


#**Text Preprocessing**

## 1.Handling Missing Data

In [9]:
# Identify missing data
missing_data = df.isnull().any(axis=1)

# Filter out rows with missing data
data_cleaned = df[~missing_data]

# Drop the index associated with the rows containing missing data
data_cleaned = data_cleaned.reset_index(drop=True)

In [10]:
data_cleaned.isnull().sum()

PositionShortText    0
LongText             0
ArticleNumber        0
Quantity             0
dtype: int64

##2.Lowercasing the Text

In [11]:
 # Convert all string columns to lowercase
data_cleaned = data_cleaned.applymap(lambda x: x.lower() if isinstance(x, str) else x)

## 3.Remove punctuation and special characters

In [12]:
import re

# Function to remove punctuation and special characters from text
def remove_punctuation_special_characters(text):
    # Define regex pattern to match any character that is not a word character or whitespace
    pattern = r'[^\w\säöüßÄÖÜ]'  # Including German special characters ä, ö, ü, ß, Ä, Ö, and Ü

    # Use regex to substitute all non-word characters and non-whitespace characters with an empty string
    cleaned_text = re.sub(pattern, '', text)
    return cleaned_text

# Remove punctuation and special characters from 'LongText' column
data_cleaned['LongText'] = data_cleaned['LongText'].apply(remove_punctuation_special_characters)

# Remove punctuation and special characters from 'PositionShortText' column
data_cleaned['PositionShortText'] = data_cleaned['PositionShortText'].apply(remove_punctuation_special_characters)



## 4.Removing Stopwords

In [13]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [14]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download stopwords list for German if not already downloaded
nltk.download('stopwords')
nltk.download('punkt')

# Get the set of German stopwords
stop_words_german = set(stopwords.words('german'))

# Function to remove stopwords from text
def remove_stopwords_german(text):
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words_german]
    return ' '.join(filtered_tokens)

# Remove stopwords from 'LongText' column
data_cleaned['LongText'] = data_cleaned['LongText'].apply(remove_stopwords_german)

# Remove stopwords from 'PositionShortText' column
data_cleaned['PositionShortText'] = data_cleaned['PositionShortText'].apply(remove_stopwords_german)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## 5.Lemmetization

In [15]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [16]:
# !pip install pattern

In [18]:
from pattern.de import lemma

# Function to lemmatize text in German
def lemmatize_text_german(text):
    tokens = text.split()  # Split text into tokens
    lemmatized_tokens = []
    for token in tokens:
        try:
            lemmatized_token = lemma(token)
            lemmatized_tokens.append(lemmatized_token)
        except StopIteration:
            # Handle StopIteration exception
            # You can choose to ignore the token or handle it differently
            pass
    lemmatized_text = ' '.join(lemmatized_tokens)  # Join lemmatized tokens back into text
    return lemmatized_text

# Lemmatize 'LongText' column in German
data_cleaned['LongText'] = data_cleaned['LongText'].apply(lemmatize_text_german)

# Lemmatize 'PositionShortText' column in German
data_cleaned['PositionShortText'] = data_cleaned['PositionShortText'].apply(lemmatize_text_german)



## 6.Tokenization

In [19]:
import nltk
from nltk.tokenize import word_tokenize

# Download NLTK resources for German language if not already downloaded
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Tokenize 'LongText' column in German
data_cleaned['LongText_tokens'] = data_cleaned['LongText'].apply(lambda x: word_tokenize(x, language='german'))

# Tokenize 'PositionShortText' column in German
data_cleaned['PositionShortText_tokens'] = data_cleaned['PositionShortText'].apply(lambda x: word_tokenize(x, language='german'))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [20]:
data_cleaned


,PositionShortText,LongText,ArticleNumber,Quantity,LongText_tokens,PositionShortText_tokens
0,zulagen schweißungen flachen ecken pvcpen sfde...,zulagen schweißungen flachen ecken innenliegen...,k141011032000,1.0,"[zulagen, schweißungen, flachen, ecken, innenl...","[zulagen, schweißungen, flachen, ecken, pvcpen..."
1,zulagen schweißungen senkrechen ecken pvcpen s...,zulagen schweißungen senkrechen ecken innenlie...,k141011032100,1.0,"[zulagen, schweißungen, senkrechen, ecken, inn...","[zulagen, schweißungen, senkrechen, ecken, pvc..."
2,zulagen schweißungen flachesen tstücken pvcpen...,zulagen schweißungen flachen tstücksen innenli...,k141011032200,1.0,"[zulagen, schweißungen, flachen, tstücksen, in...","[zulagen, schweißungen, flachesen, tstücken, p..."
3,zulagen schweißungen senkrechtesen tstücken pv...,zulagen schweißungen senkrechen tstücksen inne...,k141011032300,1.0,"[zulagen, schweißungen, senkrechen, tstücksen,...","[zulagen, schweißungen, senkrechtesen, tstücke..."
4,zulagen schweißungen flachen kreuzungen pvcpen...,zulagen schweißungen flachen kreuzungen innenl...,k141011032400,1.0,"[zulagen, schweißungen, flachen, kreuzungen, i...","[zulagen, schweißungen, flachen, kreuzungen, p..."
...,...,...,...,...,...,...
2003,baustelleneinrichtungen,einrichten unterhalten abbauen baustelleneinri...,mk,2.0,"[einrichten, unterhalten, abbauen, baustellene...",[baustelleneinrichtungen]
2004,verbauen boxgleitschienenverbauen,großflächenverbauen boxgleitschienenverbauen l...,3303,2000.0,"[großflächenverbauen, boxgleitschienenverbauen...","[verbauen, boxgleitschienenverbauen]"
2005,verbauen boxgleitschienenverbauen,großflächenverbauen boxgleitschienenverbauen l...,3303,2000.0,"[großflächenverbauen, boxgleitschienenverbauen...","[verbauen, boxgleitschienenverbauen]"
2006,baustelleneinrichtungen,einrichten unterhalten abbauen baustelleneinri...,bc,2.0,"[einrichten, unterhalten, abbauen, baustellene...",[baustelleneinrichtungen]


In [21]:
# Save the DataFrame to an Excel file
data_cleaned.to_excel('/content/drive/MyDrive/data_cleaned.xlsx', index=False)


In [22]:
data_cleaned

,PositionShortText,LongText,ArticleNumber,Quantity,LongText_tokens,PositionShortText_tokens
0,zulagen schweißungen flachen ecken pvcpen sfde...,zulagen schweißungen flachen ecken innenliegen...,k141011032000,1.0,"[zulagen, schweißungen, flachen, ecken, innenl...","[zulagen, schweißungen, flachen, ecken, pvcpen..."
1,zulagen schweißungen senkrechen ecken pvcpen s...,zulagen schweißungen senkrechen ecken innenlie...,k141011032100,1.0,"[zulagen, schweißungen, senkrechen, ecken, inn...","[zulagen, schweißungen, senkrechen, ecken, pvc..."
2,zulagen schweißungen flachesen tstücken pvcpen...,zulagen schweißungen flachen tstücksen innenli...,k141011032200,1.0,"[zulagen, schweißungen, flachen, tstücksen, in...","[zulagen, schweißungen, flachesen, tstücken, p..."
3,zulagen schweißungen senkrechtesen tstücken pv...,zulagen schweißungen senkrechen tstücksen inne...,k141011032300,1.0,"[zulagen, schweißungen, senkrechen, tstücksen,...","[zulagen, schweißungen, senkrechtesen, tstücke..."
4,zulagen schweißungen flachen kreuzungen pvcpen...,zulagen schweißungen flachen kreuzungen innenl...,k141011032400,1.0,"[zulagen, schweißungen, flachen, kreuzungen, i...","[zulagen, schweißungen, flachen, kreuzungen, p..."
...,...,...,...,...,...,...
2003,baustelleneinrichtungen,einrichten unterhalten abbauen baustelleneinri...,mk,2.0,"[einrichten, unterhalten, abbauen, baustellene...",[baustelleneinrichtungen]
2004,verbauen boxgleitschienenverbauen,großflächenverbauen boxgleitschienenverbauen l...,3303,2000.0,"[großflächenverbauen, boxgleitschienenverbauen...","[verbauen, boxgleitschienenverbauen]"
2005,verbauen boxgleitschienenverbauen,großflächenverbauen boxgleitschienenverbauen l...,3303,2000.0,"[großflächenverbauen, boxgleitschienenverbauen...","[verbauen, boxgleitschienenverbauen]"
2006,baustelleneinrichtungen,einrichten unterhalten abbauen baustelleneinri...,bc,2.0,"[einrichten, unterhalten, abbauen, baustellene...",[baustelleneinrichtungen]


In [23]:
from keras.preprocessing.text import Tokenizer

# Initialize a tokenizer
tokenizer = Tokenizer()

# Fit the tokenizer on the LongText_tokens and PositionShortText_tokens
all_tokens = data_cleaned['LongText_tokens'].tolist() + data_cleaned['PositionShortText_tokens'].tolist()
tokenizer.fit_on_texts(all_tokens)

# Convert text to sequences of numbers
data_cleaned['LongText_sequences'] = tokenizer.texts_to_sequences(data_cleaned['LongText_tokens'])
data_cleaned['PositionShortText_sequences'] = tokenizer.texts_to_sequences(data_cleaned['PositionShortText_tokens'])


#**Train-Test Split of the data**

In [24]:
from sklearn.model_selection import train_test_split

# Define features (X) and target variable (y)
X = data_cleaned[['LongText_sequences', 'PositionShortText_sequences' ]]  # Features
y = data_cleaned[['ArticleNumber','Quantity']]  # Target variable (replace 'Target_Column' with your actual target column name)

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print the shapes of the train and test sets
print("Train set shape:", X_train.shape, y_train.shape)
print("Test set shape:", X_test.shape, y_test.shape)


Train set shape: (1606, 2) (1606, 2)
Test set shape: (402, 2) (402, 2)


In [25]:
X_train.iloc[0:6]


,LongText_sequences,PositionShortText_sequences
1249,"[38, 75, 551, 205, 231, 144, 167, 168, 162, 16...","[554, 205, 1, 1216]"
261,"[124, 45, 125, 126, 127, 7, 8, 10, 7, 86, 17, ...","[120, 222, 6, 428]"
607,"[556, 619, 383, 587, 52]","[620, 1717, 990]"
973,"[124, 45, 125, 126, 127, 7, 8, 10, 7, 86, 17, ...","[120, 154, 50]"
240,"[38, 75, 399, 522, 205, 136, 7, 8, 221, 16, 66...","[381, 205, 1, 650]"
819,"[38, 75, 218, 266, 71, 8, 221, 207, 55, 196, 1...","[210, 225, 136, 400]"


In [26]:
y_train.head()

,ArticleNumber,Quantity
1249,24900100,0.21
261,40001400,5022.00
607,k462052003500,1.00
973,69910600,6820.00
240,24100000,3.00


##Storing each of it as separate variables

In [27]:
# Store X_train and X_test columns as separate variables
X_train_LongText_sequences = X_train['LongText_sequences']
X_train_PositionShortText_sequences = X_train['PositionShortText_sequences']

X_test_LongText_sequences = X_test['LongText_sequences']
X_test_PositionShortText_sequences = X_test['PositionShortText_sequences']

# Store 'Quantity' column for train and test sets
y_train_quantity = y_train['Quantity']
y_test_quantity = y_test['Quantity']

# Store 'ArticleNumber' column for train and test sets
y_train_article_number = y_train.copy()
y_test_article_number = y_test.copy()



#**Encoding The Article Number**


In [28]:
data_cleaned['ArticleNumber'] = data_cleaned['ArticleNumber'].astype(str)


In [29]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit LabelEncoder on the article numbers and transform them
data_cleaned['ArticleNumber_encoded'] = label_encoder.fit_transform(data_cleaned['ArticleNumber'])

# Print the mapping of original article numbers to encoded values
#print("Mapping of original article numbers to encoded values:")
#for original_value, encoded_value in zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)):
#    print(f"{original_value}: {encoded_value}")


In [30]:
encoded_variable_counts = data_cleaned['ArticleNumber_encoded'].value_counts()
TotalArticles = encoded_variable_counts.shape[0]
print(f"Unique Articles : {TotalArticles}")

Unique Articles : 306


In [31]:
# Group the DataFrame by 'ArticleNumber_encoded' and aggregate the indexes for each group
indexes_by_article_number = data_cleaned.groupby('ArticleNumber_encoded').apply(lambda x: list(x.index))

# Print the indexes for each article number
#for article_number, indexes in indexes_by_article_number.items():
#    print(f"Article Number {article_number}: Indexes {indexes}")


In [32]:
 # Extract long text sequences and position short text sequences from the DataFrame
long_text_sequences = data_cleaned['LongText_sequences'].tolist()
position_short_text_sequences = data_cleaned['PositionShortText_sequences'].tolist()


In [33]:
# Convert each integer sequence into a list of lists
tokenized_long_text_sequences = [[token] for token in long_text_sequences]
tokenized_position_short_text_sequences = [[token] for token in position_short_text_sequences]



In [34]:
from collections import Counter

# Assuming tokenized_long_text_sequences and tokenized_position_short_text_sequences are lists of tokenized sequences
# Remove the extra brackets around each sequence
tokenized_long_text = tokenized_long_text_sequences  # List of tokenized long text sequences
tokenized_short_text = tokenized_position_short_text_sequences  # List of tokenized short text sequences

# Concatenate tokenized sequences
all_tokenized_texts = tokenized_long_text + tokenized_short_text

# Flatten the list of tokenized sequences and handle nested lists
all_tokens = [token for sequence in all_tokenized_texts for token_list in sequence for token in token_list]

# Count the frequency of each token
token_counts = Counter(all_tokens)

# Calculate the vocabulary size
vocabulary_size = len(token_counts)

print("Vocabulary size:", vocabulary_size)


Vocabulary size: 2140


# Decoder

In [35]:
# Decode the encoded article numbers back to their original values
decoded_article_numbers = label_encoder.inverse_transform(data_cleaned['ArticleNumber_encoded'])

# Print the decoded article numbers
print("Decoded article numbers:")
print(decoded_article_numbers)


Decoded article numbers:
['k141011032000' 'k141011032100' 'k141011032200' ... '3303' 'bc' 'bc']


# Classification Regression Model

## SELF-ATTENTION Calculation

In [36]:
def scaled_dot_product_attention(q, k, v):
    """
    Args:
    q: query shape == (..., seq_len_q, depth) # NOTE: depth=dk
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable to (..., seq_len_q, seq_len_k). Defaults to None.

    Returns:
    output, attention_weights
    """
    matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)
    # scale matmul_qk. underroot d_model i.e. underroot(100)
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)
    # softmax is normalized on the last axis (seq_len_k) so that the scores add up to 1.
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)
    output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

    return output, attention_weights

## MultiHeadAttention Calculation

In [37]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model  # typically 512

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model)

    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, v, k, q):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)

        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

        # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

        concat_attention = tf.reshape(scaled_attention,
                                      (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

        return output, attention_weights